In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
#getting dataset using kaggle data
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 90% 73.0M/80.9M [00:00<00:00, 115MB/s] 
100% 80.9M/80.9M [00:00<00:00, 101MB/s]


In [ ]:
from zipfile import ZipFile
rowdata = '/content/sentiment140.zip'
with ZipFile(rowdata,'r') as updata:
  updata.extractall()
  print("finished")

finished


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


now data processing and all

In [ ]:
col_names=['target','id','date','flag','user','data']
df = pd.read_csv("/content/training.1600000.processed.noemoticon.csv",names=col_names,encoding='ISO-8859-1')
df.shape

(1600000, 6)

In [ ]:
df.head()

,target,id,date,flag,user,data
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.drop(['id','date','flag','user'],axis=1)

,target,data
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
df['target'].value_counts()
port_stem = PorterStemmer()

In [ ]:
df.replace({'target':{4:1}},inplace=True)

In [ ]:
def stemming(content):
  stemmed_cont = re.sub('[^a-zA-Z]',' ',content)
  stemmed_cont = stemmed_cont.lower()
  stemmed_cont = stemmed_cont.split()
  stemmed_cont = [port_stem.stem(word) for word in stemmed_cont if not word in stopwords.words('english')]
  stemmed_cont = ' '.join(stemmed_cont)
  return stemmed_cont

In [ ]:
df['stemmed_content'] = df['data'].apply(stemming)

In [ ]:
df.drop(['id','date','flag','user'],axis=1,inplace=True)
df.head()

,target,data,stemmed_content
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
df.drop(['data'],axis=1,inplace=True)
df.head()

,target,stemmed_content
0,0,switchfoot http twitpic com zl awww bummer sho...
1,0,upset updat facebook text might cri result sch...
2,0,kenichan dive mani time ball manag save rest g...
3,0,whole bodi feel itchi like fire
4,0,nationwideclass behav mad see


In [ ]:
X_train , X_test ,Y_train,Y_test = train_test_split(df['stemmed_content'],df['target'],test_size=0.25)

In [ ]:
print(X_train.shape,X_test.shape)

(1200000,) (400000,)


In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
machine = LogisticRegression(max_iter=2000)


In [ ]:
machine.fit(X_train,Y_train)

LogisticRegression(max_iter=2000)

In [ ]:
x_train_predction = machine.predict(X_train)
training_data_accuracy = accuracy_score(x_train_predction,Y_train)

In [ ]:
print(training_data_accuracy)

0.8043108333333333


In [ ]:
machine.score(X_test,Y_test)

0.77694

In [ ]:
x_test_predction = machine.predict(X_test)
testing_data_accuracy = accuracy_score(x_test_predction,Y_test)
print(testing_data_accuracy)

0.77694
